In [49]:
import pandas as pd
import plotly as py
import plotly.graph_objs as go
import plotly.express as px

In [85]:
df = pd.read_csv("VizTool_users.csv")
#print(myData)

This workflow is to visualize NCRMP Viz Tool user data for the United States
1. Filter the data to only include locations within the US

In [79]:
# Get data for only the United States
df = df[df['Country'] == 'United States']
# display the dataframe
#print(df)


2. Sort values in ascending order by number of users and build bar chart of US states by total number of users

In [80]:
# Sort values in ascending order by number of users
df.sort_values(by='Users', inplace=True)

# Build bar chart
fig = px.bar(df, x='Region', y='Users',
             hover_data=['Users','Engaged sessions','Engagement rate','Average engagement time'], color='Users',
             height=400)
fig.show()

3. Convert long state names to 2 letter abbreviations. Filter out entries with no state or region name.

In [81]:

stateDict = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}

df = df[df.Region != '(not set)']
print(df)

stateList = []
regionList = df['Region'].tolist()
for region in regionList:
    calculatedState = stateDict[region]
    stateList.append(calculatedState)
countList = df['Users'].tolist()
#print(countList)

           Country                Region  Users  New users  Engaged sessions  \
191  United States               Vermont      1          1                 1   
190  United States              Nebraska      1          1                 0   
189  United States                Alaska      1          1                 0   
101  United States         West Virginia      2          0                 0   
100  United States            New Mexico      2          2                 0   
99   United States                Nevada      2          2                 2   
98   United States               Montana      2          2                 1   
97   United States                 Idaho      2          2                 3   
96   United States              Arkansas      2          2                 2   
66   United States                 Maine      3          1                 3   
68   United States          Rhode Island      3          3                 2   
67   United States              Oklahoma

4. Build a US map of total users per state

In [17]:
fig = px.choropleth(df, locations=stateList, locationmode="USA-states", hover_data=['Users','Engaged sessions','Engagement rate','Average engagement time'], color=df['Users'], scope="usa")

fig.show()

5. Build map of total engaged sessions per state

In [43]:
fig = px.choropleth(df, locations=stateList, locationmode="USA-states", hover_data=['Users','Engaged sessions','Engagement rate','Average engagement time'], color=df['Engaged sessions'], scope="usa")
fig.show()

In [83]:
df.describe()
totalUsers = df['Users'].sum()
totalSessions = df['Engaged sessions'].sum()
print("TOTAL NUMBER OF USERS: " + str(totalUsers))
print("TOTAL NUMBER OF SESSIONS: " + str(totalSessions))

TOTAL NUMBER OF USERS: 1796
TOTAL NUMBER OF SESSIONS: 728


This workflow is to visualize the geographic extent of users outside of the United States.

1. Filter out United States data.
2. Create bar chart of Users by country with a region breakdown.

In [86]:
df = pd.read_csv("VizTool_users.csv")
#print(myData)

df = df[df.Country != 'United States']
#df = df[df.Region != '(not set)']

df = df[df.Users >=1]
df = df[df.Country != 'RÃ©union']
df = df[df.Country != 'TÃ¼rkiye']
df.sort_values(by='Users', inplace=True)

fig = px.bar(df, x='Country', y='Users',
             hover_data=['Users','Region','Engaged sessions','Engagement rate','Average engagement time'], color='Users',
             height=400).update_xaxes(categoryorder="total ascending")
fig.show()

3. Use country_converter to update the country names to machine readable country codes.
4. Use the country codes to generate a world map of international users.

In [92]:
import country_converter as coco
cc = coco.CountryConverter()

countryCodeList =[]
print(df)
#df = df.groupby('Country', as_index=False)['Users'].sum()
sortedData = df.groupby('Country')['Users'].sum()
sessionsPerUserDF = df.groupby('Country')['Engaged sessions per user'].sum()
engagedSessionDF = df.groupby('Country')['Engaged sessions'].sum()
engagementRateDF = df.groupby('Country')['Engagement rate'].sum()
engagementTimeDF = df.groupby('Country')['Average engagement time'].sum()
#print(engagedSessionDF)

sortedDataFrame = pd.DataFrame({'Country':sortedData.index, 'Users':sortedData.values, 'Engaged Sessions':engagedSessionDF.values, 'Engagement Rate':engagementRateDF.values, 'Average Engagement Time':engagementTimeDF.values, 'Engaged Sessions per User':sessionsPerUserDF.values})
sortedDataFrame = sortedDataFrame[sortedDataFrame.Country != 'RÃ©union']
sortedDataFrame = sortedDataFrame[sortedDataFrame.Country != 'TÃ¼rkiye']
#print(sortedDataFrame)

countryList = sortedDataFrame.Country.tolist()
#print(str(countryList))
for country in countryList:
    countryCode = cc.convert(names = country, to = 'ISO3')
    if countryCode not in countryCodeList:
        countryCodeList.append(countryCode)

numberOfUsersList = sortedDataFrame.Users.tolist()

sortedDataFrame = pd.DataFrame({'Country':countryCodeList, 'Users': numberOfUsersList, 'Engaged Sessions':engagedSessionDF.values, 'Engagement Rate':engagementRateDF.values, 'Average Engagement Time':engagementTimeDF.values, 'Engaged Sessions per User':sessionsPerUserDF.values})

fig = px.choropleth(sortedDataFrame, locations=countryCodeList,
                    hover_data=['Users','Engaged Sessions','Engagement Rate','Average Engagement Time'],
                    #color=sortedDataFrame['Users'],
                    color=sortedDataFrame['Engaged Sessions'],
                    hover_name=countryList, # column to add to hover information
                    )

fig.update_geos(
    resolution=50,
    #showcoastlines=True, coastlinecolor="RebeccaPurple",
    #showland=True, landcolor="LightGreen",
    showocean=True, oceancolor="LightBlue",
    #showlakes=True, lakecolor="Blue",
    #showrivers=True, rivercolor="Blue"
)

fig.show()


CuraÃ§ao not found in regex


                 Country              Region  Users  New users  \
117             CuraÃ§ao           (not set)      1          1   
137            Indonesia       North Sumatra      1          0   
138            Indonesia           West Java      1          1   
139            Indonesia  West Nusa Tenggara      1          0   
140            Indonesia          West Papua      1          1   
..                   ...                 ...    ...        ...   
25                 China           (not set)     11          7   
24             Australia          Queensland     12         11   
19   U.S. Virgin Islands          St. Thomas     14         12   
15        United Kingdom             England     23         23   
14           Puerto Rico            San Juan     29         26   

     Engaged sessions  Engagement rate  Engaged sessions per user  \
117                 1         1.000000                   1.000000   
137                 0         0.000000                   0.000000   


In [77]:
meanEngagementRate = sortedDataFrame['Engagement Rate'].mean()
meanEngagementRate = (meanEngagementRate)*100
medianEngagementRate = sortedDataFrame['Engagement Rate'].median()

meanEngagementTime = sortedDataFrame['Average Engagement Time'].mean()
medianEngagementTime = sortedDataFrame['Average Engagement Time'].median()

sortedDataFrame.describe()

totalUsers = sortedDataFrame['Users'].sum()
totalSessions = sortedDataFrame['Engaged Sessions'].sum()

print("TOTAL NUMBER OF USERS: " + str(totalUsers))
print("TOTAL NUMBER OF SESSIONS: " + str(totalSessions))

#print("Mean engagement rate: " + str(meanEngagementRate))

TOTAL NUMBER OF USERS: 371
TOTAL NUMBER OF SESSIONS: 181
